# Extract metrics with LMM

In [ ]:
import fitz
from dotenv import load_dotenv
from loguru import logger
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

from src.general import sleep, encode_fitz_page

load_dotenv()

In [ ]:
LLM = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.0, max_retries=2)
CHAIN = LLM | JsonOutputParser()

METRICS = [
    "net property income",
    "distribution per unit (DPU)",
    "investment properties",
    "total assets",
    "total liabilities",
    "total debts",
    "total number of units",
    "net asset value (NAV)",
    "aggregate leverage",
    "cost of debt",
    "interest cover",
    "average term to maturity / weighted average tenor of debt",
    "weighted average lease expiry (WALE)",
]


@sleep(2)
def get_response(query: str, img_data: str) -> str:
    message = HumanMessage(
        content=[
            {"type": "text", "text": query},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{img_data}"},
            },
        ],
    )

    try:
        return CHAIN.invoke([message])
    except Exception as e:
        logger.error(e)


def format_query(metric_names: list[str]) -> str:
    query = """From the given image, extract the latest value for the following metrics:
{metric_names}

If no information can be found for the metric, say "UNK". \
Output your response as a json like this:

{{
    "net property income": ...,
    "distribution per unit (DPU)": ...,
    ...
}}"""
    return query.format(metric_names=metric_names)


def compute_metrics(doc: fitz.Document) -> dict[str, str]:
    results = {m: "UNK" for m in METRICS}
    for _, page in enumerate(doc):
        metric_names = [m for m, v in results.items() if v == "UNK"]
        if len(metric_names) == 0:
            logger.info("All found")
            break

        query = format_query(metric_names)
        img_data = encode_fitz_page(page)
        res = get_response(query, img_data)
        results.update(res)

        for k, v in res.items():
            if v != "UNK":
                logger.info(f"  {k}: {v}")
    return results


def load_pdf(pdf_filepath):
    return fitz.open(pdf_filepath)

In [ ]:
pdf_filepath = ""

In [ ]:
doc = load_pdf(pdf_filepath)
results = compute_metrics(doc)